### 1. 
Import data, split and normalize

In [2]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
SEED = 42
rng = np.random.default_rng(SEED)

DTYPE = np.float32

def fit_minmax(X):
    """Fit per-feature min-max on X (2D). Returns (mins, ranges) with safe ranges."""
    mins = X.min(axis=0)
    maxs = X.max(axis=0)
    rng = maxs - mins
    rng_safe = np.where(rng > 0, rng, 1.0)  # avoid division by zero (constant features)
    return mins.astype(DTYPE), rng_safe.astype(DTYPE)

def transform_minmax(X, mins, rng_safe):
    return ((X - mins) / rng_safe).astype(DTYPE)

def inverse_minmax(X_scaled, mins, rng_safe):
    return (X_scaled * rng_safe + mins).astype(DTYPE)

def split_indices(n, val_frac=0.15, test_frac=0.15, rng=rng):
    idx = np.arange(n)
    rng.shuffle(idx)
    n_test = int(round(test_frac * n))
    n_val  = int(round(val_frac * n))
    n_train = n - n_val - n_test
    return idx[:n_train], idx[n_train:n_train+n_val], idx[n_train+n_val:]

def create_inp_out(df):
    X_list, Y_list, M_list = [], [], []  # M is mask
    for _, row in df.iterrows():
        dV_ges = float(row["dV_ges"]) / 3.6 * 1e-6
        eps_0  = float(row["eps_0"])
        phi_0  = float(row["phi_0"])
        lam    = float(row["lambda"])
        H      = float(row["H_DPZ"])
        L      = float(row["L_DPZ"])

        X_list.append([dV_ges, eps_0, phi_0])

        y = [lam, H, L]
        m = [1.0 if v != -1.0 else 0.0 for v in y]
        Y_list.append(y)
        M_list.append(m)

    X = np.asarray(X_list, dtype=DTYPE)
    Y = np.asarray(Y_list, dtype=DTYPE)
    M = np.asarray(M_list, dtype=DTYPE)
    return X, Y, M


class XYMDataset(Dataset):
    def __init__(self, X, Y, M):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.M = torch.tensor(M, dtype=torch.float32)
    def __len__(self): return self.X.shape[0]
    def __getitem__(self, i):
        return self.X[i], self.Y[i], self.M[i]

In [3]:
TRAIN_FRAC, VAL_FRAC, TEST_FRAC = 0.70, 0.15, 0.15
torch.manual_seed(SEED)
CSV_PATH = os.path.join("Input", "ye.csv")

# 1) Read original CSV
df = pd.read_csv(CSV_PATH)

# 2) Create input-output pairs
X, Y, M = create_inp_out(df)
N = X.shape[0]

# 3) Split 
i_tr, i_va, i_te = split_indices(N, VAL_FRAC, TEST_FRAC, rng)
X_train, Y_train, M_train = X[i_tr], Y[i_tr], M[i_tr]
X_val, Y_val, M_val = X[i_va], Y[i_va], M[i_va]
X_test, Y_test, M_test = X[i_te], Y[i_te], M[i_te]


# 4) Normalize only present labels in TRAIN
min_X, rng_X = fit_minmax(X_train)

# Targets: fit per-dimension using only 
tgt_min = np.zeros(3, dtype=DTYPE)
tgt_rng = np.ones(3, dtype=DTYPE)

for j in range(3):
    present = (M_train[:, j] == 1.0)
    if present.any():
        mins_j, rngs_j = fit_minmax(Y_train[present, [j]])
        tgt_min[j] = mins_j.item()
        tgt_rng[j] = rngs_j.item()
    else:
        # No labels for this target in train: keep default min=0, rng=1 (won't be used)
        tgt_min[j] = 0.0
        tgt_rng[j] = 1.0

# -------------------
# Transform splits (normalize only where target labels exist)
# -------------------
def transform_split_minmax(X, Y, M, min_X, rng_X, tgt_min, tgt_rng):
    Xn = transform_minmax(X, min_X, rng_X)
    Yn = Y.copy()
    for j in range(3):
        present = (M[:, j] == 1.0)
        Yn[present, j] = (Yn[present, j] - tgt_min[j]) / tgt_rng[j]
    return Xn, Yn, M

Xn_train, Yn_train, Mn_train = transform_split_minmax(X_train, Y_train, M_train, min_X, rng_X, tgt_min, tgt_rng)
Xn_val, Yn_val, Mn_val = transform_split_minmax(X_val, Y_val, M_val, min_X, rng_X, tgt_min, tgt_rng)
Xn_test, Yn_test, Mn_test = transform_split_minmax(X_test, Y_test, M_test, min_X, rng_X, tgt_min, tgt_rng)



### 2. 
Create model

In [8]:
EPS = 1e-12  # numerical safety
import torch.nn as nn
HIDDEN = [128, 128, 128]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MultiHead(nn.Module):
    def __init__(self, in_dim=3, hidden=HIDDEN):
        super().__init__()
        layers = []
        d = in_dim
        for h in hidden:
            layers += [nn.Linear(d, h), nn.ReLU()]
            d = h
        self.trunk = nn.Sequential(*layers)
        self.head_lambda = nn.Linear(d, 1)   # -> λ
        self.head_geom   = nn.Linear(d, 2)   # -> [H, L]

    def forward(self, x):
        z = self.trunk(x)
        lam = self.head_lambda(z)  # (B,1)
        HL  = self.head_geom(z)    # (B,2)
        return lam, HL

def masked_mse(pred, target, mask, eps=EPS):
    """
    pred, target, mask: (B,D)
    Computes mean square error only over entries with mask=1.
    """
    se = (pred - target)**2 * mask
    denom = mask.sum().clamp_min(eps)
    return se.sum() / denom

def evaluate(dloader, model, tgt_min, tgt_rng, device=DEVICE):
    """Return MAE/MSE on the original (denormalized) scale for each target [λ,H,L]."""
    model.eval()
    mae = np.zeros(3, dtype=np.float64)
    mse = np.zeros(3, dtype=np.float64)
    count = np.zeros(3, dtype=np.float64)

    with torch.no_grad():
        for xb, yb, mb in dloader:
            xb = xb.to(device)
            yb = yb.to(device)        # normalized targets
            mb = mb.to(device)

            lam_pred, HL_pred = model(xb)
            pred = torch.cat([lam_pred, HL_pred], dim=1)  # (B,3)

            pred_np = pred.cpu().numpy()
            y_np    = yb.cpu().numpy()
            m_np    = mb.cpu().numpy().astype(bool)

            # inverse min-max per dim
            for j in range(3):
                p = pred_np[:, j] * tgt_rng[j] + tgt_min[j]
                t = y_np[:, j]    * tgt_rng[j] + tgt_min[j]
                m = m_np[:, j]
                if m.any():
                    diff = p[m] - t[m]
                    mae[j] += np.abs(diff).sum()
                    mse[j] += (diff**2).sum()
                    count[j] += m.sum()

    mae = np.where(count>0, mae/count, np.nan)
    mse = np.where(count>0, mse/count, np.nan)
    return mae, mse


model = MultiHead(in_dim=3, hidden=HIDDEN).to(DEVICE)

### 3. 
Train the model

In [9]:
EPOCHS = 2000
BATCH_SIZE = 6
LR = 1e-3
opt = torch.optim.Adam(model.parameters(), lr=LR)

ds_tr = XYMDataset(Xn_train, Yn_train, Mn_train)
ds_va = XYMDataset(Xn_val, Yn_val, Mn_val)
ds_te = XYMDataset(Xn_test, Yn_test, Mn_test)
dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
dl_va = DataLoader(ds_va, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
dl_te = DataLoader(ds_te, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

valid_counts = Mn_train.sum(axis=0)  # (3,)
w_lam = 1.0 / max(valid_counts[0], 1.0)
w_geo = 1.0 / max(valid_counts[1] + valid_counts[2], 1.0)


for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0
    for xb, yb, mb in dl_tr:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)  # normalized targets
        mb = mb.to(DEVICE)

        lam_pred, HL_pred = model(xb)          # (B,1), (B,2)
        lam_t = yb[:, [0]]                      # (B,1)
        HL_t  = yb[:, 1:3]                      # (B,2)
        m_lam = mb[:, [0]]
        m_geo = mb[:, 1:3]

        loss_lam = masked_mse(lam_pred, lam_t, m_lam)
        loss_geo = masked_mse(HL_pred,  HL_t,  m_geo)

        loss = w_lam * loss_lam + w_geo * loss_geo

        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item() * xb.size(0)

    if epoch % 100 == 0 or epoch == 1:
        mae_va, mse_va = evaluate(dl_va, model, tgt_min, tgt_rng)
        print(f"Epoch {epoch:4d} | train_loss ~ {total_loss/len(ds_tr):.4e} | "
              f"VA-MAE [λ,H,L]= {mae_va} | VA-MSE [λ,H,L]= {mse_va}")

Epoch    1 | train_loss ~ 2.7647e-02 | VA-MAE [λ,H,L]= [0.11655438 0.02465879 0.12494775] | VA-MSE [λ,H,L]= [0.01917257 0.00080122 0.02712737]
Epoch  100 | train_loss ~ 1.4183e-03 | VA-MAE [λ,H,L]= [0.05648899 0.00309906 0.03317885] | VA-MSE [λ,H,L]= [1.16157407e-02 1.12452615e-05 1.72498525e-03]
Epoch  200 | train_loss ~ 7.2619e-04 | VA-MAE [λ,H,L]= [0.05383626 0.00481178 0.03670461] | VA-MSE [λ,H,L]= [1.15111880e-02 4.37874009e-05 2.31883357e-03]
Epoch  300 | train_loss ~ 5.3578e-04 | VA-MAE [λ,H,L]= [0.04856056 0.00613807 0.03448819] | VA-MSE [λ,H,L]= [8.80592960e-03 7.51765841e-05 1.67715494e-03]
Epoch  400 | train_loss ~ 5.3901e-04 | VA-MAE [λ,H,L]= [0.04447352 0.00737133 0.02948564] | VA-MSE [λ,H,L]= [0.00704298 0.00010547 0.00143589]
Epoch  500 | train_loss ~ 2.0632e-04 | VA-MAE [λ,H,L]= [0.05236634 0.00600959 0.04194733] | VA-MSE [λ,H,L]= [9.75995927e-03 5.61886677e-05 2.94035214e-03]
Epoch  600 | train_loss ~ 7.1047e-04 | VA-MAE [λ,H,L]= [0.0448761  0.00924852 0.03733893] | VA

### 4.
Evaluate results

In [16]:
# -------------------
# Final evaluation on TEST
# -------------------
mae_te, mse_te = evaluate(dl_te, model, tgt_min, tgt_rng)
mape_te = mae_te / tgt_rng * 100.0  # in percent
print("TEST  MAPE [λ,H,L]:", mape_te, '%')
print("TEST  MAE [λ,H,L]:", mae_te)
print("TEST  MSE [λ,H,L]:", mse_te)

TEST  MAPE [λ,H,L]: [2.1753852  6.35497911 8.38830774] %
TEST  MAE [λ,H,L]: [0.00538611 0.00350795 0.03179169]
TEST  MSE [λ,H,L]: [4.07779887e-05 1.51146596e-05 1.34482840e-03]
